## Readability Challenge Machine Learning

In this machine learning project, we are going to be predicting the difficulty of different texts based on some important features. Since we only have the textual information, we are going to need to create new features and also tokenize the existing words into different words and sentences to understand some useful features. 

In addition to this, we would also find correlation between different features that we have created and see how much of an impact they make when we are performing the machine learning analysis and predictions respectively. 

In [ ]:
import numpy as np                       ##Used for numerical computations 
import pandas as pd                      ##Used for reading the data
import seaborn as sns                    #Used for plotting with more features
import matplotlib.pyplot as plt          #Used for plotting 
from nltk.corpus import stopwords        ##This is used to plot the number of stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize        ##This is used to divide the overall text data to tokens and sentences
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer   ##Using the vectorizer to convert values
import tqdm                        ##Used for measuring the time it takes to get the things done 
import re                           ##Standard library for reading and substituting the word expressions 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
import nltk                                ##Used for the natural language processing tasks 
import missingno as msno                      ## Used to plot the missing values that are present in our data
from wordcloud import WordCloud               ##It is used to plot the frequency of the words which determines their size
try:
    import plotly.express as px               ##This library is used for interactive visualization 
    from plotly import graph_objects as go    ##We also have to use this along with plotly to get interactive visualization
except:
    !pip install plotly
import warnings                       ##We are going to filer some warnings and remove them when we try to import the libraries
warnings.filterwarnings("ignore")

Here, we are going to be just looking at the present directory and see the path that we are currently at so that we can understand where our datasets are present. 

In [ ]:
pwd()

As can be seen below, the train path and the test path are created which would later be used to read the datasets based on where they are present and the path. 

In [ ]:
##Path used for training and testing data 
TRAIN_PATH = 'Readability datasets/'
TEST_PATH = 'Readability datasets/'

Now is the time to store the .csv files in different variables so that we later can use them for the machine learning analysis and predictions. 

In [ ]:
##Reading the training data, testing data and sample values that we are going to be understanding and using in the long term. 

df_train = pd.read_csv(TRAIN_PATH + 'train.csv', low_memory = False)
df_test = pd.read_csv(TEST_PATH + 'test.csv', low_memory = False)
df_sample = pd.read_csv(TEST_PATH + 'sample_submission.csv', low_memory = False)

It is always a good idea to check the memory usage so that there won't be any issues when performing operations that require more RAM. Sometimes when we don't check the usage, there might be a case where the amount of RAM that is being consumed for the machine learning operations far exceeds the capacity of our RAM, leading to errors in the code. Therefore, it would be good to understand the total amount of RAM that is being used respectively. 

In [ ]:
##info is used to check the total number of null values and also the feature types respectively
df_train.info(memory_usage = 'deep')

Let us now perform the same operation for the test set to understand the data and also see the different columns where there are NULL values present. 

In [ ]:
df_test.info(memory_usage = 'deep')

Performing the same operation for the sample dataframe to see the type of columns that we must include when doing the submission. 

In [ ]:
df_sample.info(memory_usage = 'deep')

## Reading the Head of Training Data

Now, we see from the below that there are some columns that contain NULL values such as "url_legal" and "license" columns respectively. One observation from the table below is that there is only a text that we are given along with the "target" which is what we are going to predict for the future text along with "standard_error" respectively. 

Since there are not many features rather than just the text given, we have to be creating new features that helps us to get the best predictions in the test set. Therefore, we have to featurize based on the text and create now columns and append in our existing dataset which ensures that we get a model that has a good accuracy and low mean square error. 

In [ ]:
##Reading the head of the dataframe that we are working on. 
df_train.head()

Taking a look at the sample data where we must be giving the specific ID along with the predictions of our machine learning and deep learning algorithms and submit it. 

In [ ]:
df_sample.head()

## Using Missingno to plot the missing null values
It would be a good idea to understand the missing values that are present in our data with the help of a figure. Below, we are going to be using the missingno library which gives us the missing values that are present based on different columns respectively. We see that there are a few missing values that are present in the columns "url_legal" and "license" respectively as marked below. 

In addition, we can also find that there are no missing values present for "excerpt", "target" and "standard_error" respectively. 

In [ ]:
##Using the missingno library that is used to get the list of missing values through a figure. 
msno.matrix(df_train, color = (0.01, 0.75, 0.75))

## Creating Useful Functions
It would be a good idea to create functions and add some of the arguments so that we don't have to repeat the code again. In addition, we can just change the arguments and it would be applied entirely to the block that of code that is present in the function. Therefore, it would be really convenient to create functions along with useful arguments that we might change when we are implemented them in the future of our code. 

Below, we have defined a function called "plot_histogram" where it takes in the dataframe along with the column that we are interested and a few other arguments and plot a histogram of our configuration. 

In [ ]:
def plot_histogram(dataframe: pd.DataFrame, column: str, x_title = None, y_title = None, edge_color = 'black', 
                   color = 'Green', total_bins = 20, font_size = 10):
    """
    In this function, we are going to plot a histogram for the dataframe that is given
    along with some modifications and different features of histogram plot
    """
    
    fig, ax = plt.subplots(figsize = (10, 5))    ##This is used to create many plots depending on which we choose to use respectively.
    ax.hist(dataframe[column], bins = total_bins, edgecolor = edge_color, color = color)
    ax.set_title(f'Histogram Distribution of {column} values', fontsize = font_size + 5)
    ax.set_xlabel(column, fontsize = font_size)
    ax.set_ylabel("Total Number of Observations", fontsize = font_size)
    plt.show()

## Histogram of Target Variable
Since most of the parameters are set to default values, those values are taken into consideration when plotting the histogram plots respectively. Since we want to look at the column called 'target' in our data and it's distribution, we are using that column and giving it to the function so that it plots respectively. 

We see that the output is evenly distributed with the mean around -1. In addition, we find that there are very few values that are present that are greater than 1 or less than -3 as can be seen from the below plot. 

In [ ]:
##Plotted the histogram of the variable that we are going to be predicting respectively. 
plot_histogram(df_train, column = 'target')

## Histogram of Standard_Error Values
In the same light, let us now look at the standard error distribution by using the function that we have created to plot the histogram. Now we have changed the color to see the changes and how they would be applied to the block of functions. 

There are a lot of values that are present around 0.5 than the other regions. We understand that there are very few values where the standard error is 0.6 or more. In addition, there are very few values where the standard error is below 0.4 respectively.  

In [ ]:
#Using the histogram values and we are going to plot the standard error in orange as can be seen below
plot_histogram(df_train, column = 'standard_error', color = 'orange')

In [ ]:
##Reading the text that we want to see and looking at how the sentences are framed.
df_train['excerpt'].iloc[1]

In [ ]:
##Replacing a few characters that may not be useful for our machine learning predictions respectively. 
df_train['excerpt'].apply(lambda text: text.replace('\n', ' ')).iloc[1]

In [ ]:
df_train['excerpt'] = df_train['excerpt'].apply(lambda text: text.replace('\n', ' '))
df_test['excerpt'] = df_test['excerpt'].apply(lambda text: text.replace('\n', ' '))

In [ ]:
df_train.head()

In [ ]:
df_train['excerpt'].iloc[1]

## Wordcloud function

We are going to be creating a wordcloud function that is important for machine learning. We are going to be using that function to create words and understand their frequency based on the size of those words respectively. Since it would be a good idea to use wordcloud in a function so that we could use the same block of code again.

From the observation, we see that there are a few words such as "One" and "said" that are frequently occurring in our dataset. We were able to understand this with the help of the size of the words that are present in the wordcloud that indicate their frequency respectively. We also consider "." as a word and we see that it is also frequently occurrring respectively. There are some words that don't occur as much as the above mentioned words such as "country" and "never". 

Therefore, we were able to get a good understanding of the wordcloud figure and see the frequency of the occurence of the words based on the size in the figure given by the wordcloud. 

In [ ]:
def wordcloud_function(df, title, column, fontsize = 15, 
                       figsize = (10, 10), width = 300, height = 300):
    """
    This function ensures that we get the size of the words based on their occurance 
    and we are going to use a figure to plot it in this function respectively. 
    """
    total_text = ''.join([text for text in df[column]])
    wordcloud = WordCloud(width = width, height = height).generate(total_text)
    plt.figure(figsize = figsize)
    plt.imshow(wordcloud)
    plt.title(title, fontsize = fontsize)
    plt.show()
wordcloud_function(df_train, title = "Wordcloud function", column = "excerpt")
    

Now it is time to create a preprocessing function that would take into account the text and convert it in the form where the machine learning and deep learning models could use quite easily. Therefore, we are going to take that data and ensure that we understand how the machine learning models are going to perform in the future. 

In preprocessing function, we would just replace the punctuations and other things with only the text present in the excerpt. Later, we would lowercase the words and then remove the stopwords. Once the stopwords are removed, we are going to be lemmatizing those words (reducing words to their base form) so that their stem is retained along with taking the context. We would return the output as a list. 

In [ ]:
len(df_train)

In [ ]:
len(df_train) * 0.7

In [ ]:
len(df_train) * 0.75

## Creating preprocessing function
It is now time to understand the data and convert the essay values to different set of values that could be given to different vectorizers that are important for machine learning respectively. We would be performing various operations when we are creating this function. We would take the text and replace all the values with only alphabets. Later, we are going to lowercase the text and then tokenize and then, remove the stopwords that do not add a lot of meaning to the text. Finally, a dataframe is constructed and returned as output from the function. 

In [ ]:
def preprocessing_function(df):
    """
    This function takes into consideration the dataframe and extracts the text.
    In addition, it makes modifications to the text and converts it to a simpler form
    for machine learning processing respectively."""
    
    text_list = []
    for text in tqdm(df['excerpt'].values):
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        text = nltk.word_tokenize(text)
        [word for word in text if not word in set(stopwords.words("english"))]
        lemmatizer = nltk.WordNetLemmatizer()
        text = [lemmatizer.lemmatize(word) for word in text]
        text = " ".join(text)
        text_list.append(text)
    text_list = pd.Series(text_list)
    text_list.column = ['Converted_text']
    return text_list
        

We are going to store the output that is given by the preprocessing function into a variable and later use it. 

In [ ]:
preprocessed_text = preprocessing_function(df_train)

In [ ]:
preprocessed_text_test = preprocessing_function(df_test)

## Defining get_useful_features functions

It is now time to get the useful features that are important for machine learning. We would have to be creating new features that would help the machine learning models to get the best predictions for the difficulty of the text. 

Taking into consideration the excerpt and stopwords, we are going to be creating new feautres such as total number of words, sentence length, overall change in the text length and other features that are important for getting the machine learning outputs. The function would return the final dataframe that contains all the preprocessed output along with the newly created features that are important for machine learning. 

In [ ]:
def get_useful_features(df, stop_words):
    """
    The function would take the dataframe and stopwords and then, convert the excerpts into different features
    such as the number of sentences, words and the lenght of the lemmas created along with the overall preprocessed
    essay length."""
    sentences = []
    num_of_words = []
    sent_length = []
    word_length = []
    lemma_length = []
    num_of_lemmas = []
    preprocessed_essay_length = []
    initial_text_length = []
    num_of_sentences = []
    text_shortage = []
    
    for text in tqdm(df['excerpt'].values):
        
        initial_length = len(text)
        initial_text_length.append(initial_length)
        num_sentences = len(sent_tokenize(text))
        num_of_sentences.append(num_sentences)
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        text = word_tokenize(text)
        num_words = len(text) 
        num_of_words.append(num_words)
        sent_length.append(num_words/num_sentences)
        word_length.append(initial_length/num_words)
        text = [word for word in text if not word in stop_words]
        lemmatizer = nltk.WordNetLemmatizer()
        text = [lemmatizer.lemmatize(word) for word in text]
        #print(text)
        num_lemmas = len(text)
        num_of_lemmas.append(num_lemmas)
        text = " ".join(text)
        #print(text)
        preprocessed_essay_length_value = len(text)
        preprocessed_essay_length.append(preprocessed_essay_length_value)
        #print(preprocessed_essay_length)
        #print(num_lemmas)
        lemma_length.append(preprocessed_essay_length_value/num_lemmas)
        
        text_shortage.append(preprocessed_essay_length_value/initial_length)
        
    final_df = pd.concat([pd.Series(sent_length), pd.Series(num_of_words),
                             pd.Series(word_length), pd.Series(lemma_length),
                             pd.Series(num_of_sentences), pd.Series(initial_text_length),
                             pd.Series(num_of_lemmas), pd.Series(preprocessed_essay_length),
                             pd.Series(text_shortage)], axis = 1)
    final_df.columns = ["sentence_length", "num_of_words", "word_length",
                           "lemma_length", "num_of_sentences",
                           "initial_text_length", "num_of_lemmas",
                           "preprocessed_essay_length", "text_shortage"]
    
    return final_df
        

We are now going to store those values into a variable that we can use to check the values respectively. 

In [ ]:
final_df = get_useful_features(df_train, stop_words = set(stopwords.words("english")))

In [ ]:
final_df_test = get_useful_features(df_test, stop_words = set(stopwords.words("english")))

We have created a dataframe and we see that there are different features that we are going to be giving to the machine learning models for predictions. We have converted the text in the form of different useful features such as the number of words, average word and sentence length, number of lemmas created and preprocessed essay length. Overall, this could be given to the machine learning models for prediction where the target which is nothing but the difficulty of the text would be predicted using machine learning. 

In [ ]:
final_df.head()

## Pairplot of Different Features 

It would be a good idea to see the pairplots which are nothing but a group of scatterplots where the values are spread based on the 2 features that are considered at hand. By looking at this pairplot, one would understand if there is any correlation between features and their values and spread respectively. 

When we get an almost a linear spread between features, we can understand that there is either a positive correlation or a negative correlation between the features at hand. 

Based on the observation, we see that there is a good correlation between number of words and intial text length. There is also a good positive correlation between the number of words and the text shortage metric that we have created. In addition, there is also a positive correlation between the lemma word average length and the overall average length of word before lemmatization. This tells us that based on how long the word is, the lemma form of the word would almost have identical size respectively. 

One more thing to observe based on the text values is that there is a slight negative correlation between the initial text length and the difficulty of the text. We can understand this to be true as when there are a lot of words in a sentence, it becomes easy to understand the text as the author usually would elaborate the points that he/she made and this ensures that there is clarity in the text and it being less difficult. More observations could be made using the pairplot function in seaborn. For now, we have made a good amount of useful observations that are important when doing the machine learning analysis. 

In [ ]:
df_for_plotting = pd.concat([final_df, df_train["target"]], axis = 1)
ax = sns.pairplot(data = df_for_plotting, plot_kws = dict(color = "maroon"))
plt.show()

In [ ]:
np.zeros_like((3, 3))

## Heatmap of Correlation Matrix

When we are using .corr(), we are going to get the correlation values for different features in a form of a table. Since interpreting a table is quite difficult especially with float values present, it would be a good idea to visualize the data and color the points based on their values.

This is done with the aid of heatmap where based on the values present, they would change the intensity of their color. Below is a heatmap plotted that gives us a good idea about the correlation between different features that are important for machine learning. 

By observing the values below, one could get an understand there is a slight negative correlation between lemma_length and the difficulty of the text. In addition, we also see a negative correlation between average word length and the text's difficulty respectively. We see that there is also a clear negative correlation between sentence length and number of sentences. This means that as the number of sentences increase, there is a higher possibility that there can be a high sentence length. However, this does not mean that correlation is equal to causation. But based on the information and the context, we might assure that there is causation between features respectively. 

In [ ]:
correlation = df_for_plotting.corr()
plt.figure(figsize = (10, 10))
sns.heatmap(correlation, annot = True, cmap = 'Oranges', linewidths = 1,
           annot_kws = {"weight": "bold", "fontsize": 10})

## Scatterplot function

It would be a good idea to create scatterplot function that ensures that we are able to change the parameters that are important for machine learning. We see that we might give the default parameters that are important for the scatterplot. However, we have the freedom to change the parameters as and when needed when calling the function.

Since the function would only be used for plotting rather than returning any output, there is no return in a function. 

In [ ]:
def scatterplot_function(df: pd.DataFrame, X, Y, figsize = (10, 10), color = 'orange'):
    plt.figure(figsize = figsize)
    sns.scatterplot(data = df, x = X, y = Y, color = color)
    plt.title(f'Scatterplot between {X} and {Y}')
    plt.show()

## Scatterplot between num_of_sentences and target

By looking at the heatmap above, we have considered the correlation between features and seen if there is any relationship between the features.

Now, we would be focusing on the scatterplot between 'num_of_sentences' and 'target' respectively. Below, we see that there is a slight positive relationship between the output and the "num_of_sentences" feature. This could help us understand that there might be a change that when there is increase in the number of sentences, there is a possibility of the difficulty of the text would also increase. But let us not jump to the conclusion as correlation might not always be equal to causation. Hence, let us now explore more features which would ensure that we understand them and we are going to be using them. 

In [ ]:
scatterplot_function(df_for_plotting, X = 'num_of_sentences', Y = 'target', figsize = (7, 7))

## Scatterplot between 'preprocessed_essay_length' and 'num_of_lemmas'

Now, we are going to plot between 'preprocessed_essay_length' and 'num_of_lemmas' as we have seen that their correlation value is about 0.86 respectively. Therefore, we have plotted the scatterplot to understand the values respectively. We see that there is a good correlation between the features as there is not much scatter between the 2 features that we have considered.

Lemmas are the words that are created after preprocessing the actual words and replacing those words with their stems but also considering the context at which those words appear. This ensures that we get the right stems that later could be vectorized and given to the machine learning models for processing and prediction respectively. 


In [ ]:
scatterplot_function(df_for_plotting, X = 'preprocessed_essay_length', Y = 'num_of_lemmas', figsize = (7, 7), color = 'maroon')

In [ ]:
scatterplot_function(df_for_plotting, X = 'word_length', Y = 'text_shortage', figsize = (7, 7), color = 'green')

In [ ]:
scatterplot_function(df_for_plotting, X = 'num_of_sentences', Y = 'sentence_length', figsize = (7, 7), color = 'black')

In [ ]:
type(df_train['excerpt'])

In [ ]:
df_for_plotting.head()

## Creating new function that generates more features

We are going to create a function that would create more features such as counting the number of commas, semicolons and other important features that are important for machine learning analysis. We would have to create empty lists of these values and we are going to concat those by converting them into series and then, return a new dataframe respectively. 

With the help of this function, we have created new dataframe which contains the useful columns that are important for machine learning respectively. We would be performing the feature analysis and thise ensures that we are going to be getting the best results on the test set respectively. 

At last, we are going to concat those values that are important for machine learning and this would ensure that we get the best results in the test set respectively. We are going to be taking those values and this ensures that we are getting the best results on the test set. 

In [ ]:
def generate_more_features(df: pd.DataFrame):
    """
    This function would create a dataframe of different useful features
    that are important for machine learning predictions respectively.
    """
    commas = []
    semicolon = []
    exclamations = []
    questions = []
    quotes = []
    periods = []
    longest_word = []
    
    for i in range(len(df)):
        
        #word_len = []
        text = df['excerpt'].iloc[i]
        commas.append(text.count(","))
        semicolon.append(text.count(";"))
        exclamations.append(text.count("!"))
        questions.append(text.count("?"))
        quotes.append(text.count('"'))
        periods.append(text.count('.'))
        word_len = [len(w) for w in text.split(" ")]
        longest_word.append(np.max(word_len))
        
    df_with_features =pd.concat((pd.Series(commas), pd.Series(semicolon), pd.Series(exclamations),
                               pd.Series(questions), pd.Series(quotes), pd.Series(periods),
                                pd.Series(longest_word)), axis = 1)
    df_with_features.columns = ["num_of_commas", "num_of_semicolons", "num_of_explamations",
                                "num_of_questions", "num_of_quotes", "num_of_periods", 
                                "longest_word"]
                                
    return df_with_features

We would be using those functions and we are going to be taking those values in a variable respectively. 

In [ ]:
df_with_more_features = generate_more_features(df_train)

In [ ]:
df_with_more_features_test = generate_more_features(df_test)

Reading the head of the dataframe, we see that there are different columns created along with the numerical representation respectively. Therefore, we have created a new dataframe in the above function and we have created some important features that could be used for machine learning analysis. 

In [ ]:
df_with_more_features.head()

We are now going to check whether there are any null values that are present in the newly created dataframe. We see below that there are some values that are null present in our dataframe. Therefore, we have to take our time to remove those null values so that we are going to be using the things that we are sure that the more we are learning in the long term. 

In [ ]:
any(df_with_more_features.isnull())

We are going to be creating the dataframe where there are null or not values that we are going to be using. 

In [ ]:
df_with_more_features.isnull()

We are going to be taking a look at the null values and we are going to understand the different machine learning information respectively. 

In [ ]:
df_with_more_features.info()

We are going to read the head of the dataframe and see the values and how they are spread out. 

In [ ]:
df_with_more_features.head()

In [ ]:
df_train['excerpt'].iloc[140]

In [ ]:
df_train['excerpt'].iloc[143]

In [ ]:
df_for_plotting = pd.concat([df_with_more_features, df_train['target']], axis = 1)

## Pairplot of Other Features

Understanding some of the pairplots, we are going to use them to see how the values are spread. We could see one positive correlation between the difficulty of the text and the "num_of_periods" that we have in our data. This means that more the number of sentences, there is a higher change for the text to be difficult to a certain extent. 

In addition to this, we also see that there is a relationship between the number of quotes and the difficulty of the text. We see quite a linear relationship between the features as can be seen. 

There is also a positive relationship between the number of questions and the difficulty of the text respectively. Therefore, by looking at the pairplot, we see that there are some features that are quite useful when doing the machine learning analysis respectively.

In [ ]:
sns.pairplot(df_for_plotting, plot_kws = dict(color = 'green'))
plt.show()


## Heatmap between num_of_quotes and num_of_questions

Since we have created new features, let us see how these features are related to each other and the output variable respectively. We make some interesting observations by looking at the heatmap below. 

1. The feature num_of_explanations is quite related to num_of_quotes as can be seen in the heatmap below. 
2. We see that the num_of_questions are quite related to the num_of_quotes as well. 
3. There is a negative correlation between the num_of_semicolons and the num_of_periods.
4. num_of_quotes and num_of_periods are quite positively correlated with the difficulty of the text. 
5. There is also a negative correlation between the length of the longest word and the difficulty of the text. 

With the help of heatmap, therefore, we were able to make some interesting observations above. 

In [ ]:
df_correlation = df_for_plotting.corr()
plt.figure(figsize = (10, 10))
sns.heatmap(df_correlation, annot = True, cmap = 'Oranges', linewidths = 1,
           annot_kws = {"weight": "bold", "fontsize": 10})

In [ ]:
df_with_more_features.info()

In [ ]:
generate_more_features(df_train)

In [ ]:
df_train['excerpt_length'] = df_train.excerpt.apply(len)
df_test['excerpt_length'] = df_test.excerpt.apply(len)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.head()

In [ ]:
df_train.head()

In [ ]:
def Count_Vectorizer(df: pd.DataFrame):
    vectorizer = CountVectorizer()
    vectorizer.fit(df['excerpt'])
    converted_vector = vectorizer.transform(df['excerpt'])
    return converted_vector

In [ ]:
Count_Vectorizer(df_train)

In [ ]:
from nltk.corpus import stopwords

## Creating Decontracted function 
We are going to be creating this function which would take the actual subtext values and convert to forms that are easily accessible to the reader. Some of the words can be seen below from the function where the short forms are replaced with more useful texts respectively. This function is created so that the output from this could be given to other function that we are going to create just below this function. 

In [ ]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    """
    This function would convert some short letters into the forms that one 
    could easily understand respectively.
    """
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

## Creating preprocessing_function
It is now time to create the preprocessing function that takes into account the dataframe and it would be removing the stopwords from the text. Later, it would be replacing those complex words that we have seen above with the easy words that are easier to preprocess and understand. Later, lemmatization is done which would replace the words into simpler forms so that they could be given to different machine learning and deep learning models to generate the output respectively. 

In [ ]:
def preprocessing_function(df):
    """
    In this function, we are going to be performing the basic preprocessing 
    which is needed before giving to different vectorizers. This ensures that
    we are getting the best output values respectively.
    """
    
    stopwords_new = set(stopwords.words('english'))
    text_list = []
    for i in tqdm(range(len(df['excerpt']))):
        text = df['excerpt'].iloc[i]
        text = decontracted(text)
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        text = [word for word in text.split(' ') if not word in stopwords_new]
        lemmatizer = nltk.WordNetLemmatizer()
        text = [lemmatizer.lemmatize(word) for word in text]
        text_list.append(' '.join(text))
    return text_list


We are going to take the values that are given by the function and store it in a variable as can be seen below. 

In [ ]:
df_train_preprocessed = preprocessing_function(df_train)

Similar process is applied to the test set so that we might make predictions with the best machine learning or deep learning model that we would be getting as the output. 

In [ ]:
df_test_preprocessed = preprocessing_function(df_test)

Taking a look at the head of the preprocessed that we got from the function and stored in this variable. 

In [ ]:
preprocessed_text.head()

In [ ]:
df_with_more_features.head()

In [ ]:
df_for_plotting.head()

In [ ]:
final_df.head()

## Concatenating the dataframes

We are going to be concatenating the dataframes that we have created previously. Taking some important features and the overall output values, we are later going to be generating the predictions that are important for machine learning outputs respectively. 

In [ ]:
df_complete = pd.concat((preprocessed_text, df_with_more_features, final_df), axis = 1)

We are also going to be doing the same thing for the test set. Considering the test set, we have to later apply the overall process that we have created for the training data. When we are done with the output values, we are going to be taking those new values which would ensure that we get the best output values respectively. 

In [ ]:
df_complete_test = pd.concat((preprocessed_text_test, df_with_more_features_test, final_df_test), axis = 1)

We would be looking at the dataframe we got above and looking at some of the columns that we have taken into consideration. 

In [ ]:
df_complete.head()

We are going to also see how the test set values are generated. 

In [ ]:
df_complete_test.head()

In [ ]:
preprocessed_text.head()

In [ ]:
vectorizer = TfidfVectorizer()
array_output_values = vectorizer.fit_transform(preprocessed_text)

In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df_train['excerpt'])
df_train_converted = vectorizer.transform(df_train['excerpt'])

In [ ]:
df_train_converted

In [ ]:
df_train_converted

In [ ]:
df_complete.head()

In [ ]:
df_complete_test.head()

In [ ]:
df_complete_important_features = df_complete.iloc[:, 1:]
df_complete_important_features_test = df_complete_test.iloc[:, 1:]

In [ ]:
df_complete_important_features.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

## Creating min_max_scaler function

With the help of this function, the training and the test values are converted to scaled outputs where the minimum value is 0 and the maximum value is 1 respectively. In other words, this function would just be converting those high values to lie between 0 and 1. 

In [ ]:
def min_max_scaler(df_train, df_test):
    """
    This function performs the scaling operation by taking into account the train and test set respectively. It is going to 
    convert the values that are present in the data to lie between 0 and 1 respectively.
    """
    scaler = MinMaxScaler()
    scaler.fit(df_train)
    df_scaled = scaler.transform(df_train)
    df_scaled_test = scaler.transform(df_test)
    return df_scaled, df_scaled_test

## Creating Tfidf_Vectorier function
This function would convert the training and the test values to Tfidf vectors that are important for machine learning analysis in the future respectively. Therefore, the text document is converted into a group of vectors that is easy for the machine learning and deep learning models to perform well on the training and the test data.

We have to note that the machine learning models would be working with only the numerical data rather than any other sort of data such as text and other forms of data. Hence, steps must be taken to ensure that the values that are given to the machine learning models are numerical in nature. 

In [ ]:
def Tfidf_Vectorizer(df_train, df_test):
    """
    This function would take the training data and the test data and convert those values which would be in the form of text 
    to tfidf values that could be later used for machine learning analysis.
    """
    vectorizer = TfidfVectorizer()
    df_vectorized_train = vectorizer.fit_transform(df_train)
    df_vectorized_test = vectorizer.transform(df_test)
    return df_vectorized_train, df_vectorized_test

## Creating Count_Vectorizer function 
In the count vectorizer function, the original text document both for the training and the test sets are taken and we are going to be converting those values Bag-Of-Words (BOW) representation that would be later given to the machine learning models for prediction. 

In [ ]:
def Count_Vectorizer(df_train, df_test):
    """
    This function would take the training data and the test data and convert those values which would be in the form of text to 
    bag of words representation or count vectorized values which are later used for machine learning prediction.
    """
    vectorizer = CountVectorizer()
    df_vectorized_train = vectorizer.fit_transform(df_train)
    df_vectorized_test = vectorizer.transform(df_test)
    return df_vectorized_train, df_vectorized_test

In [ ]:
df_complete_important_features
df_complete_important_features_test


In [ ]:
df_train_converted

In [ ]:
df_scaled, df_scaled_test = min_max_scaler(df_complete_important_features, df_complete_important_features_test)

## Creating Vectorizer_decision function
This function would give the user whether he/she should be using the Count Vectorizer or the Tfidf Vectorizer that is being used for machine learning. Therefore, they have the freedom to select the type of vectorizer which might be given to the dataframe which would later be used for predictions. 
The Vectorizer decision would use the above functions which we have created and this would be the main function that would take into account all the values that are present respectively. 

In [ ]:
def Vectorizer_decision(df_train, df_test):
    print("Please enter the vectorizer that you would like to use for your data")
    print("Please select from the following options")
    print("1. Count Vectorizer")
    print("2. Tfidf Vectorizer")
    choice = int(input("Enter 1 or 2:"))
    type(choice)
    if choice != 1 and choice != 2:
        choice = input("Please select only from the listed options")
        vectorized_train = 0
        vectorized_test = 0
    elif choice == 1:
        print("Count Vectorizer Selected")
        vectorized_train, vectorized_test = Count_Vectorizer(df_train.iloc[:, 0], df_test.iloc[:, 0])
    else:
        print("Tfidf Vectorizer Selected")
        vectorized_train, vectorized_test = Tfidf_Vectorizer(df_train.iloc[:, 0], df_test.iloc[:, 0])
    return vectorized_train, vectorized_test

Storing the vectorizer outputs in different variables respectively. 

In [ ]:
df_vectorized, df_vectorized_test = Vectorizer_decision(df_complete.copy(), df_complete_test.copy())

Since the output that we got is a sparse matrix after giving to the vectorizers, it is important to convert those values to numpy vectors before performing the machine learning tasks. 

In [ ]:
df_vectorized = df_vectorized.toarray()
df_vectorized_test = df_vectorized_test.toarray()

In [ ]:
df_scaled

In [ ]:
X = np.concatenate((df_vectorized, df_scaled), axis = 1)
X_test = np.concatenate((df_vectorized_test, df_scaled_test), axis = 1)
y = df_train['target'].values

In [ ]:
len(X)

In [ ]:
len(X_test)

In [ ]:
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size = 0.3, random_state = 50)

In [ ]:
def neural_network(X_train, y_train, Validation_data = None, metrics = ['mean_squared_error', 'mean_absolute_error'], 
                   activation = 'relu', input_shape = (22690, ), optimizer = 'adam', loss = 'mean_squared_error',
                  epochs = 10, batch_size = 64):
    """
    We are defining a neural network function that takes into account a different set of parameters
    that are needed to build the machine learning model and we are also giving different values
    and it would be working with different parameters and we are able to give those values to our 
    deep learning models and we are going to return the output given by the model respectively.
    """
    
    model = Sequential()
    model.add(Dense(500, activation = activation, input_shape = input_shape))
    model.add(Dense(100, activation = activation))
    model.add(Dense(50, activation = activation))
    model.add(Dense(10, activation = activation))
    model.add(Dense(5, activation = activation))
    model.add(Dense(1))
    model.compile(loss = loss, metrics = metrics, optimizer = optimizer)
    if Validation_data:
              model.fit(x = X_train, y = y_train, validation_data = Validation_data, epochs = epochs, batch_size = batch_size)
    else:
              model.fit(x = X_train, y = y_train, epochs = epochs, batch_size = batch_size)
    return model 

In [ ]:
model = neural_network(X_train = X_train, y_train = y_train, Validation_data = (X_cv, y_cv))

In [ ]:
model.predict(X_test)

## TFIDF Vectorizer

We are going to be using the Tfidf vectorizer that would give us a good understanding of the performance of the neural network model. 

In [ ]:
df_vectorized, df_vectorized_test = Vectorizer_decision(df_complete, df_complete_test)

In [ ]:
df_vectorized = df_vectorized.toarray()
df_vectorized_test = df_vectorized_test.toarray()

In [ ]:
len(df_vectorized)

In [ ]:
X = np.concatenate((df_vectorized, df_scaled), axis = 1)
X_test = np.concatenate((df_vectorized_test, df_scaled_test), axis = 1)
y = df_train['target'].values

In [ ]:
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size = 0.3, random_state = 50)

In [ ]:
model = neural_network(X_train = X_train, y_train = y_train, Validation_data = (X_cv, y_cv))

In [ ]:
def actual_vs_predictions_dataframe(dataframe1, dataframe2, column1 = 'predicted', column2 = 'actual'):
    dataframe1 = pd.DataFrame(pd.Series(dataframe1), columns = [column1])
    dataframe2 = pd.DataFrame(pd.Series(dataframe2), columns = [column2])
    dataframe_concatenated = pd.concat((dataframe1, dataframe2), axis = 1)
    return dataframe_concatenated

In [ ]:
def regression_plot_function(dataframe, fig_size = (10, 10), color = 'teal'):
    plt.figure(figsize = fig_size)
    sns.regplot(data = dataframe, y = 'predicted', x = 'actual', color = color, marker = 'o')
    plt.title("Comparision of predicted values and the actual values", fontsize = 20)
    plt.show()


In [ ]:
y_cv_predictions = model.predict(X_cv)

In [ ]:
final_dataframe = actual_vs_predictions_dataframe(y_cv_predictions.flatten(), y_cv)

In [ ]:
regression_plot_function(final_dataframe)

In [ ]:
print("The mean squared error of neural network model for cross-validation data is {}".format(mean_squared_error(y_cv_predictions, y_cv)))
print("The mean absolute error of neural network model for cross-validation data is {}".format(mean_absolute_error(y_cv_predictions, y_cv)))

## Linear Regression Machine Learning Model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_cv_predictions = model.predict(X_cv)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(y_cv_predictions, y_cv)

In [ ]:
print("The mean squared error of linear regression model for cross-validation data is {}".format(mean_squared_error(y_cv_predictions, y_cv)))
print("The mean absolute error of linear regression modle for cross-validation data is {}".format(mean_absolute_error(y_cv_predictions, y_cv)))

In [ ]:
first_dataframe = pd.DataFrame(pd.Series(y_cv), columns = ['y_actual'])
second_dataframe = pd.DataFrame(pd.Series(y_cv_predictions), columns = ['y_predictions'])

In [ ]:
final_dataframe = pd.concat((first_dataframe, second_dataframe), axis = 1)

In [ ]:
final_dataframe.head()

In [ ]:
#What are some things that we are going to be using in the long term. 